### 예제 데이터 로드

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

#### 데이터 살펴보기

In [2]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [6]:
import pandas as pd

print('target 클래스 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print()
print('target 클래스의 이름들 \n',pd.Series(news_data.target_names))

target 클래스 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64

target 클래스의 이름들 
 0                  alt.atheism
1                comp.graphics
2      comp.os.ms-windows.misc
3     comp.sys.ibm.pc.hardware
4        comp.sys.mac.hardware
5               comp.windows.x
6                 misc.forsale
7                    rec.autos
8              rec.motorcycles
9           rec.sport.baseball
10            rec.sport.hockey
11                   sci.crypt
12             sci.electronics
13                     sci.med
14                   sci.space
15      soc.religion.christian
16          talk.politics.guns
17       talk.politics.mideast
18          talk.politics.misc
19          talk.religion.misc
dtype: object


In [7]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

- 뉴스그룹 기사의 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등의 다양한 정보를 가지고 있음

### 데이터 전처리

#### 내용을 제외하고 제목 등의 다른 정보 제공

In [26]:
# subset='train'으로 학습용 데이터 추출, remove=('headers','footers','quotes')로 내용만 추출

train_news=fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'),random_state=156)

X_train = train_news.data
y_train = train_news.target

# subset='test'로 테스트 데이터 추출, remove=('headers','footers','quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'),random_state=156)

X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0}, 테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314, 테스트 데이터 크기 7532


#### 카운트 기반의 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가


**테스트 데이터는 반드시 학습 데이터를 이용해 fit()이 수행된 CountVectorizer 객체를 이용해 테스트 데이터 변환**

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization 으로 피처 벡터화 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# 학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피터 벡처화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape : ',X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape :  (11314, 101631)


- 학습 데이터를 CountVectorizer로 피처를 추출한 결과 11,314 개의 문서에 단어가 101,631개가 나온것

##### 로지스틱 회귀 적용

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')


# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
pred =  lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 예측 정확도 : {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 예측 정확도 : 0.607


#### TF-IDF 기반의 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred =  lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 예측 정확도 : {0:.3f}'.format(accuracy_score(y_test,pred)))

TF-IDF Vectorized Logistic Regression 예측 정확도 : 0.674


- TF-IDF 가 단순 카운트 기반보다 훨씬 높은 예측 정확도를 제공함
- 일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에서 TF-IDF가 좋은 예측 결과를 제공함

### 머신러닝 모델 성능 향상

#### 최상의 피처 전처리 : 다양한 파라미터 제공

In [31]:
# stop words 필터링 추가, ngram을 기본 (1,1) 에서 (1,2)로 변경해 피처 벡터화 적용
tfidf_vect = TfidfVectorizer(stop_words = 'english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression(max_iter=500)
lr_clf.fit(X_train_tfidf_vect,y_train)
pred =  lr_clf.predict(X_test_tfidf_vect)
print('CountVectorized Logistic Regression 예측 정확도 : {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 예측 정확도 : 0.692


In [32]:
from sklearn.model_selection import GridSearchCV

# 최적 C값 도출 튜닝 수행. CV는 3 폴드 세트 설정
params = {'C':[0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter : ',grid_cv_lr.best_params_)

# 최적 C 값으로 학습된 grid_cv로 예측 및 정확도 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도 : {0:.3f}'.format(accuracy_score(y_test,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter :  {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도 : 0.701
